## Qiskit Aqua Gradient Framework

In [ ]:
import numpy as np
from qiskit.aqua.operators import X, I 
from qiskit.aqua.operators.gradients import Gradient, NaturalGradient, QFI, Hessian

### First Order Gradients

In [ ]:
a = Parameter('a')
b = Parameter('b')
q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(a, q[0])
qc.rx(b, q[0])

#### Gradients w.r.t. Measurement Operator Coefficients

Given a parameterized quantum state `|ψ(θ)〉 = V(θ)|ψ〉` with input state `|ψ〉` and parametrized Ansatz `V(θ)`, and an Operator `O(ω)` we want to compute 
```math
d⟨ψ(θ)|O(ω)|ψ(θ)〉/ dω.
```

In [ ]:
# Instantiate the Hamiltonian observable
coeff_0 = Parameter('c_0')
coeff_1 = Parameter('c_1')
H = coeff_0 * X + coeff_1 * Z
# Combine the Hamiltonian observable and the state
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)
# Define the coefficients w.r.t. we want to compute the gradient
gradient_coeffs = [coeff_0, coeff_1]
# Convert the operator and the gradient target coefficients into the respective operator
grad = Gradient.convert(op, gradient_coeffs)
# Define the values to be assigned to the parameters
value_dict = {coeff_0: 0.5, coeff_1: -1, a: np.pi / 4, b: np.pi}
# Assign the parameters and evaluate the gradient
grad_result = grad.assign_parameters(value_dict).eval()
print('Gradient ', grad_result)

#### Gradients w.r.t. Measurement Quantum State Parameters

Given a parameterized quantum state `|ψ(θ)〉 = V(θ)|ψ〉` with input state `|ψ〉` and parametrized Ansatz `V(θ)`, and an Operator `O(ω)` we want to compute 
``` math
d⟨ψ(θ)|O(ω)|ψ(θ)〉/ dθ,
```
respectively, the gradient w.r.t. the probabilities of observing an output state `|i〉`
``` math
dp(|i〉) = d⟨ψ(θ)|i〉⟨i|ψ(θ)〉/ dθ = d|⟨i|ψ(θ)〉|^2 / dθ 
```
Here, the operators are set to `O_i = |i〉⟨i|`.
To simplify this computation, we measure the state `|ψ(θ)〉` and evaluate the gradients for the observed states `|i〉` (the rest is set to 0) instead of evaluating all `2^n` possible projectors. 

In [16]:
# Define the Hamiltonian with fixed coefficients
H = 0.5 * X - 1 * Z
# Define the parameters w.r.t. we want to compute the gradients
params = [a, b]

NameError: name 'X' is not defined

##### Parameter Shift Parameters
Given a Hermitian operator $g$ with two unique eigenvalues $\pm r$ which acts as generator for a parameterized quantum gate $$G(\theta)= e^{-i\theta g}$$. Then, quantum gradients can be computed by using eigenvalue $r$ dependent shifts to parameters. All parameterized qiskit gates from ---TODO--- can be shifted with $\pi/2$
,
``` math
d⟨ψ(θ)|O(ω)|ψ(θ)〉/ dθ ≈  (⟨ψ(θ+π/2)|O(ω)|ψ(θ+π/2)〉 - ⟨ψ(θ-π/2)|O(ω)|ψ(θ-π/2))*2
```

In [ ]:
# Define the values to be assigned to the parameters
value_dict = { a: np.pi / 4, b: np.pi}

# Combine the Hamiltonian observable and the state
state_grad_op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)
state_grad = Gradient().convert(operator=state_grad_op, params=params, method='param_shift')
# Assign the parameters and evaluate the gradient
state_grad_result = state_grad.assign_parameters(value_dict).eval()
print('State gradient computed with parameter shift', state_grad_result)

prob_grad_op = CircuitStateFn(primitive=qc, coeff=1.)
prob_grad = Gradient().convert(operator=prob_grad_op, params=params, method='param_shift')
# Assign the parameters and evaluate the gradient
prob_grad_result = prob_grad.assign_parameters(value_dict).eval()
print('Probability gradient computed with parameter shift', prob_grad_result)

##### Finite Difference Parameters

Unlike the other methods, finite difference gradients are numerical estimations rather than analytical values.
This implementation employs a central difference approach.
``` math
d⟨ψ(θ)|O(ω)|ψ(θ)〉/ dθ ≈  (⟨ψ(θ+ε)|O(ω)|ψ(θ+ε)〉 - ⟨ψ(θ-ε)|O(ω)|ψ(θ-ε))/2ε,
```

In [14]:
# Define the values to be assigned to the parameters
value_dict = { a: np.pi / 4, b: np.pi}

# Combine the Hamiltonian observable and the state
state_grad_op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)
state_grad = Gradient().convert(operator=state_grad_op, params=params, method='finite_diff')
# Assign the parameters and evaluate the gradient
state_grad_result = state_grad.assign_parameters(value_dict).eval()
print('State gradient computed with finite difference', state_grad_result)

prob_grad_op = CircuitStateFn(primitive=qc, coeff=1.)
prob_grad = Gradient().convert(operator=prob_grad_op, params=params, method='finite_diff')
# Assign the parameters and evaluate the gradient
prob_grad_result = prob_grad.assign_parameters(value_dict).eval()
print('Probability gradient computed with finite difference', prob_grad_result)

NameError: name 'Gradient' is not defined

##### Linear Combination of Unitaries Parameters
Variance linear combination, fewer circuits add qubit

In [ ]:
# Define the values to be assigned to the parameters
value_dict = { a: np.pi / 4, b: np.pi}

# Combine the Hamiltonian observable and the state
state_grad_op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)
state_grad = Gradient().convert(operator=state_grad_op, params=params, method='lin_comb')
# Assign the parameters and evaluate the gradient
state_grad_result = state_grad.assign_parameters(value_dict).eval()
print('State gradient computed with linear combination of unitaries', state_grad_result)

prob_grad_op = CircuitStateFn(primitive=qc, coeff=1.)
prob_grad = Gradient().convert(operator=prob_grad_op, params=params, method='lin_comb')
# Assign the parameters and evaluate the gradient
prob_grad_result = prob_grad.assign_parameters(value_dict).eval()
print('Probability gradient computed with linear combination of unitaries', prob_grad_result)

#### Natural Gradient



In [ ]:
# Define the values to be assigned to the parameters
value_dict = { a: np.pi / 4, b: np.pi}

# Combine the Hamiltonian observable and the state
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)
# A regularization method can be chosen, e.g. ridge or lasso with automatic parameter search
nat_grad = NaturalGradient().convert(operator=op, params=params, method='lin_comb', regularization='ridge')
# Assign the parameters and evaluate the gradient
nat)grad_result = nat_grad.assign_parameters(value_dict).eval()
print('Natural gradient computed with linear combination of unitaries', nat_grad_result)

### Second Order Gradients

#### Gradients w.r.t. Measurement Operator Coefficients

#### Gradients w.r.t. Measurement Quantum State Parameters

##### Parameter Shift Parameters

##### Linear Combination of Unitaries Parameters

### QFI

#### Full QFI
Using linear combination of unitaries

#### Block-diagonal Approximation
Without working qubits
Requires unrolling into Pauli rotations and unparameterized Gates

#### Diagonal Approximation
Without working qubits
Requires unrolling into Pauli rotations and unparameterized Gates

### Application Examples

#### VQE with first and second order gradient based optimization

In [ ]:
from qiskit.aqua.operators import I, X, Z
from qiskit.circuit import QuantumCircuit, ParameterVector
from scipy.optimize import minimize

h2_hamiltonian = -1.05 * (I ^ I) + 0.39 * (I ^ Z) - 0.39 * (Z ^ I) - 0.01 * (Z ^ Z) + 0.18 * (X ^ X)
h2_energy = -1.85727503


# Define the Ansatz
wavefunction = QuantumCircuit(2)
params = ParameterVector('theta', length=8)
it = iter(params)
wavefunction.ry(next(it), 0)
wavefunction.ry(next(it), 1)
wavefunction.rz(next(it), 0)
wavefunction.rz(next(it), 1)
wavefunction.cx(0, 1)
wavefunction.ry(next(it), 0)
wavefunction.ry(next(it), 1)
wavefunction.rz(next(it), 0)
wavefunction.rz(next(it), 1)

def fun(params):
    param_dict = dict(zip(wavefunction.parameters, params)) @ wavefunction
    op = ~StateFn(h2_hamiltonian) 
    return op.assign_parameters(param_dict).eval()

# TODO jac and hessian getting callable
result = minimize(fun, x0, , method='dogleg', jac=None, hess=None)

print('VQE:', result, 'Reference:', h2_energy)

#### Gibbs state preparation using Variational Quantum Imaginary Time Evolution (VarQITE)


In [17]:
from qiskit.circuit.library import RealAmplitudes

# Temperature
T = 5

# Evolution time
t =  1/(2*T)

# Define the model Hamiltonian
H = (0.3 * Z^Z + 0.2 * Z^I - 0.5 * I ^ Z) ^ I^I
# Instantiate the model ansatz
depth = 1
ansatz = RealAmplitudes(4, reps=depth, entanglement = 'sca')
qr = ansatz.qregs[0]
for i in range(int(len(qr)/2)):
    ansatz.cx(qr[i], qr[i+int(len(qr)/2)])
    
# Define the Hamiltonian as observable w.r.t. the wavefunction generated by the Ansatz    
op = ~StateFn(H) @ ansatz

# Define the discretization grid of the time steps
num_time_steps = 100
time_steps = np.linspace(0, t, num_time_steps)

# Initialize the Ansatz parameters
param_values = np.zeros(2 * H.num_qubits * 2 * (depth + 1))
for j in range(2 * H.num_qubits * 2 * depth, int(len(param_values) - 2 * H.num_qubits -1), 2):
    param_values[j] = np.pi / 2.

# Convert the operator that holds the Hamiltonian and ansatz into a NaturalGradient operator 
nat_grad = NaturalGradient.convert(op, ansatz.ordered_parameters)

# Propagate the Ansatz parameters step by step according to the explicit Euler method
for step in time_steps:
    param_dict = dict(zip(ansatz.ordered_parameters, param_values)
    param_values -= t/num_time_steps * nat_grad.assign_parameter(param_dict).eval()
  

SyntaxError: invalid syntax (<ipython-input-17-9c499b9d403f>, line 8)

#### QBMs? - see if time allows